In [29]:
import pandas as pd

In [36]:
df = pd.read_csv('../raw_data/lula.csv')

In [37]:
df.tail()

,url,date,content,id,reply_count,retweet_count,like_count,quote_count,lang,username,displayname,description,verified,created,followers_count,friends_count,location,protected,profile_img
5,https://twitter.com/WiltonAlmSPO/status/148830...,2022-01-31 23:59:51+00:00,@mauriciocarjr @Nilsonhandebol @LulaOficial #SDV,1488301028470665226,1,0,1,0,und,WiltonAlmSPO,Wilton Almeida 🏳️‍🌈🇾🇪,Poupem-me de passar raiva hoje. Bem-vindos e n...,False,2021-09-17 19:42:02+00:00,3246,2210,NaN,False,https://pbs.twimg.com/profile_images/143895150...
6,https://twitter.com/maryy_mo/status/1488301022...,2022-01-31 23:59:49+00:00,@acervommes @_pxle lula ladrão de periquita,1488301022455943172,1,0,0,0,pt,maryy_mo,Mari Cruz🐣,♌ @flamengo\nPETISTA SAFADA insta::moraismaryh16,False,2016-06-03 14:39:01+00:00,975,817,NaN,False,https://pbs.twimg.com/profile_images/151328378...
7,https://twitter.com/MarcosM02671644/status/148...,2022-01-31 23:59:45+00:00,@IvaniRocha10 @neuzaortiz @SF_Moro Gostemos ou...,1488301006479888390,0,0,1,0,pt,MarcosM02671644,Marcos Marciano,NaN,False,2020-08-05 20:02:19+00:00,386,1822,NaN,False,https://pbs.twimg.com/profile_images/129110282...
8,https://twitter.com/Odracimartinsc1/status/148...,2022-01-31 23:59:44+00:00,@senadorhumberto @LulaOficial O povo quer Lula...,1488300999706038274,0,0,1,0,pt,Odracimartinsc1,Odracimartinsc1,"palmeirense, patriota, liberal conservador!",False,2019-07-30 18:37:37+00:00,3301,4715,"Osasco, Piemonte",False,https://pbs.twimg.com/profile_images/152312651...
9,https://twitter.com/joaopeguima/status/1488300...,2022-01-31 23:59:43+00:00,quem anima ir pra posse do lula em brasilia no...,1488300998267490304,10,0,21,2,pt,joaopeguima,peguima,família de direita de acordo com a Bíblia sagr...,False,2016-01-13 21:14:03+00:00,2347,468,NaN,False,https://pbs.twimg.com/profile_images/152309613...


In [ ]:
df.shape

(10, 19)

In [43]:
df.iloc[7][2]

'@IvaniRocha10 @neuzaortiz @SF_Moro Gostemos ou nao,Lula e bom de debate;O ex-juiz ia passar vergonha...'